# Notebook 2: Coleta e Análise de Dados ATRICON

**Objetivo**: Coletar e analisar os dados de transparência pública dos 10 municípios selecionados a partir do Programa Nacional de Transparência Pública (PNTP) da ATRICON.

**Fonte de Dados**: 
- ATRICON - Associação dos Membros dos Tribunais de Contas do Brasil
- Programa Nacional de Transparência Pública (PNTP) - Ciclo 2024
- Arquivo: avaliacoes_pntp_2024.csv

## 1. Importação de Bibliotecas

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## 2. Carregamento dos Dados

In [9]:
# Carregar lista de municípios selecionados no Notebook 1
df_selecionados = pd.read_csv('transparencia-municipal-analise/Data/processed/municipios_selecionados_final.csv')
print(f"Municípios selecionados: {len(df_selecionados)}")
print("\nLista de municípios:")
for idx, row in df_selecionados.iterrows():
    print(f"  {idx+1}. {row['Municipio']}")

Municípios selecionados: 10

Lista de municípios:
  1. Porto Velho
  2. Ji-Paraná
  3. Vilhena
  4. Ariquemes
  5. Cacoal
  6. Jaru
  7. Rolim de Moura
  8. Pimenta Bueno
  9. Guajará-Mirim
  10. Ouro Preto do Oeste


In [35]:
# Carregar dados de avaliações ATRICON
import pandas as pd

df_atricon = pd.read_csv('../Data/raw/Atricon/avaliacoes_pntp_2024/avaliacoes_pntp_2024.csv', sep=';')

df_atricon.head()


,ano_exercicio,questionario_id,entidade_id,entidade,ibge,municipio,capital,uf,poder,esfera,...,nivel_avaliacao,indice_validacao,essenciais_validacao,nivel_validacao,indice_revisao,essenciais_revisao,nivel_revisao,indice_final,essenciais_final,nivel_final
0,2024,9768,12,Prefeitura Municipal de Abel Figueiredo,1500131,Abel Figueiredo,False,PA,E,M,...,Diamante,96.50,100.00,Diamante,NaN,NaN,NaN,96.50,100.00,Diamante
1,2024,12808,2591,Prefeitura Municipal de João Lisboa,2105500,João Lisboa,False,MA,E,M,...,Diamante,87.67,100.00,Ouro,NaN,NaN,NaN,87.67,100.00,Ouro
2,2024,11084,5641,Câmara Municipal de Águas Frias,4200556,Águas Frias,False,SC,L,M,...,Ouro,68.58,76.19,Intermediário,NaN,NaN,NaN,68.58,76.19,Intermediário
3,2024,14850,5909,Câmara Municipal de Araquari,4201307,Araquari,False,SC,L,M,...,Ouro,66.92,76.19,Intermediário,NaN,NaN,NaN,66.92,76.19,Intermediário
4,2024,13318,10352,Câmara Municipal de São José do Piauí,2210201,São José do Piauí,False,PI,L,M,...,Intermediário,39.24,90.48,Básico,NaN,NaN,NaN,39.24,90.48,Básico


## 3. Filtrar Dados de Rondônia

In [ ]:
# Filtrar apenas municípios de Rondônia
df_atricon_ro = df_atricon[df_atricon['uf'] == 'RO'].copy()
print(f"Total de avaliações em Rondônia: {len(df_atricon_ro)}")

# Filtrar apenas Poder Executivo Municipal (Prefeituras)
df_atricon_ro_exec = df_atricon_ro[df_atricon_ro['poder'] == 'E'].copy()
print(f"Avaliações de Prefeituras em Rondônia: {len(df_atricon_ro_exec)}")

# Visualizar amostra
print("\nAmostra dos dados:")
df_atricon_ro_exec[['municipio', 'indice_avaliacao', 'nivel_avaliacao', 
                     'indice_final', 'nivel_final']].head(10)

## 4. Integrar Dados ATRICON com Municípios Selecionados

In [ ]:
# Preparar dados para merge
# Normalizar nomes de municípios para facilitar o merge
df_selecionados['Municipio_Norm'] = df_selecionados['Municipio'].str.upper().str.strip()
df_atricon_ro_exec['Municipio_Norm'] = df_atricon_ro_exec['municipio'].str.upper().str.strip()

# Realizar merge
df_analise = pd.merge(
    df_selecionados,
    df_atricon_ro_exec[['Municipio_Norm', 'ibge', 'indice_avaliacao', 'essenciais_avaliacao', 
                        'nivel_avaliacao', 'indice_validacao', 'essenciais_validacao',
                        'nivel_validacao', 'indice_final', 'essenciais_final', 'nivel_final']],
    on='Municipio_Norm',
    how='left'
)

print(f"\nMunicípios com dados ATRICON: {df_analise['indice_final'].notna().sum()}")
print(f"Municípios sem dados ATRICON: {df_analise['indice_final'].isna().sum()}")

# Verificar municípios sem dados
if df_analise['indice_final'].isna().sum() > 0:
    print("\nMunicípios sem dados de transparência:")
    print(df_analise[df_analise['indice_final'].isna()]['Municipio'].tolist())

df_analise.head()

## 5. Análise Descritiva dos Indicadores de Transparência

In [ ]:
# Estatísticas descritivas dos índices
print("=" * 100)
print("ESTATÍSTICAS DESCRITIVAS - INDICADORES DE TRANSPARÊNCIA ATRICON")
print("=" * 100)

print("\nÍNDICE DE AVALIAÇÃO:")
print(df_analise['indice_avaliacao'].describe())

print("\nÍNDICE DE VALIDAÇÃO:")
print(df_analise['indice_validacao'].describe())

print("\nÍNDICE FINAL:")
print(df_analise['indice_final'].describe())

print("\nESSENCIAIS FINAL:")
print(df_analise['essenciais_final'].describe())

In [ ]:
# Distribuição dos níveis de transparência
print("\n" + "=" * 100)
print("DISTRIBUIÇÃO DOS NÍVEIS DE TRANSPARÊNCIA (ATRICON)")
print("=" * 100)

if 'nivel_final' in df_analise.columns:
    nivel_counts = df_analise['nivel_final'].value_counts()
    print("\nNível Final:")
    for nivel, count in nivel_counts.items():
        print(f"  {nivel}: {count} município(s)")
else:
    print("Coluna 'nivel_final' não encontrada")

## 6. Ranking de Transparência dos Municípios Selecionados

In [ ]:
# Criar ranking por índice final de transparência
df_ranking_transp = df_analise.sort_values('indice_final', ascending=False).reset_index(drop=True)
df_ranking_transp['Ranking_Transp'] = df_ranking_transp.index + 1

print("=" * 120)
print("RANKING DE TRANSPARÊNCIA - MUNICÍPIOS SELECIONADOS")
print("=" * 120)
print(df_ranking_transp[[
    'Ranking_Transp', 'Municipio', 'Populacao', 'PIB_Milhares_Reais',
    'indice_final', 'essenciais_final', 'nivel_final'
]].to_string(index=False))

## 7. Visualizações

In [ ]:
# Gráfico de barras - Índice Final de Transparência
fig, ax = plt.subplots(figsize=(14, 7))

df_plot = df_ranking_transp.sort_values('indice_final', ascending=True)

# Definir cores por nível
nivel_cores = {
    'Diamante': '#9b59b6',
    'Ouro': '#f39c12',
    'Prata': '#95a5a6',
    'Intermediário': '#3498db',
    'Básico': '#e74c3c',
    'Inicial': '#e67e22',
    'Inexistente': '#c0392b'
}

cores = [nivel_cores.get(nivel, '#7f8c8d') for nivel in df_plot['nivel_final']]

bars = ax.barh(df_plot['Municipio'], df_plot['indice_final'], 
               color=cores, edgecolor='black', linewidth=1.2)

ax.set_xlabel('Índice Final de Transparência (%)', fontsize=12)
ax.set_title('Índice de Transparência ATRICON - Top 10 Municípios de Rondônia (2024)', 
            fontsize=14, fontweight='bold')
ax.set_xlim(0, 100)
ax.grid(True, alpha=0.3, axis='x')

# Adicionar valores e níveis nas barras
for i, (idx, row) in enumerate(df_plot.iterrows()):
    if pd.notna(row['indice_final']):
        ax.text(row['indice_final'] + 2, i, 
               f"{row['indice_final']:.2f}% - {row['nivel_final']}", 
               va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('ranking_transparencia_atricon.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico salvo: ranking_transparencia_atricon.png")

In [ ]:
# Gráfico de dispersão - População vs Índice de Transparência
fig, ax = plt.subplots(figsize=(12, 8))

# Filtrar apenas municípios com dados
df_plot_scatter = df_analise[df_analise['indice_final'].notna()].copy()

scatter = ax.scatter(df_plot_scatter['Populacao'], 
                     df_plot_scatter['indice_final'],
                     s=df_plot_scatter['PIB_Milhares_Reais'] / 10000,  # Tamanho proporcional ao PIB
                     alpha=0.6, 
                     c=range(len(df_plot_scatter)),
                     cmap='viridis',
                     edgecolor='black',
                     linewidth=1.5)

# Adicionar labels
for idx, row in df_plot_scatter.iterrows():
    ax.annotate(row['Municipio'], 
               (row['Populacao'], row['indice_final']),
               xytext=(10, 5), textcoords='offset points',
               fontsize=9, alpha=0.8)

ax.set_xlabel('População (habitantes)', fontsize=12)
ax.set_ylabel('Índice Final de Transparência (%)', fontsize=12)
ax.set_title('Relação entre População e Transparência Pública\n(Tamanho da bolha = PIB)', 
            fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 100)

plt.tight_layout()
plt.savefig('populacao_vs_transparencia.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico salvo: populacao_vs_transparencia.png")

In [ ]:
# Gráfico de dispersão - PIB vs Índice de Transparência
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(df_plot_scatter['PIB_Milhares_Reais'], 
                     df_plot_scatter['indice_final'],
                     s=df_plot_scatter['Populacao'] / 500,  # Tamanho proporcional à população
                     alpha=0.6, 
                     c=range(len(df_plot_scatter)),
                     cmap='plasma',
                     edgecolor='black',
                     linewidth=1.5)

# Adicionar labels
for idx, row in df_plot_scatter.iterrows():
    ax.annotate(row['Municipio'], 
               (row['PIB_Milhares_Reais'], row['indice_final']),
               xytext=(10, 5), textcoords='offset points',
               fontsize=9, alpha=0.8)

ax.set_xlabel('PIB Municipal (milhares de reais)', fontsize=12)
ax.set_ylabel('Índice Final de Transparência (%)', fontsize=12)
ax.set_title('Relação entre PIB e Transparência Pública\n(Tamanho da bolha = População)', 
            fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 100)

plt.tight_layout()
plt.savefig('pib_vs_transparencia.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico salvo: pib_vs_transparencia.png")

## 8. Análise de Correlação

In [ ]:
# Calcular correlações
df_corr = df_analise[['Populacao', 'PIB_Milhares_Reais', 'PIB_Per_Capita', 
                       'indice_final', 'essenciais_final']].copy()
df_corr = df_corr.dropna()

correlacao = df_corr.corr()

print("=" * 100)
print("MATRIZ DE CORRELAÇÃO - INDICADORES SOCIOECONÔMICOS E TRANSPARÊNCIA")
print("=" * 100)
print(correlacao)

# Visualizar matriz de correlação
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlacao, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            fmt='.3f', vmin=-1, vmax=1)
ax.set_title('Correlação entre Indicadores Socioeconômicos e Transparência', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('matriz_correlacao_completa.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nGráfico salvo: matriz_correlacao_completa.png")

## 9. Comparação dos Indicadores ATRICON

In [ ]:
# Comparar índice de avaliação, validação e final
df_indices = df_analise[['Municipio', 'indice_avaliacao', 'indice_validacao', 'indice_final']].copy()
df_indices = df_indices.dropna()
df_indices = df_indices.sort_values('indice_final', ascending=False)

print("=" * 100)
print("COMPARAÇÃO DOS ÍNDICES ATRICON")
print("=" * 100)
print(df_indices.to_string(index=False))

# Gráfico de comparação
fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(df_indices))
width = 0.25

bars1 = ax.bar(x - width, df_indices['indice_avaliacao'], width, 
               label='Índice de Avaliação', alpha=0.8, edgecolor='black')
bars2 = ax.bar(x, df_indices['indice_validacao'], width, 
               label='Índice de Validação', alpha=0.8, edgecolor='black')
bars3 = ax.bar(x + width, df_indices['indice_final'], width, 
               label='Índice Final', alpha=0.8, edgecolor='black')

ax.set_xlabel('Município', fontsize=12)
ax.set_ylabel('Índice (%)', fontsize=12)
ax.set_title('Comparação dos Índices ATRICON por Município', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(df_indices['Municipio'], rotation=45, ha='right')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 100)

plt.tight_layout()
plt.savefig('comparacao_indices_atricon.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nGráfico salvo: comparacao_indices_atricon.png")

## 10. Resumo e Conclusões

In [ ]:
print("=" * 100)
print("RESUMO DA ANÁLISE DE TRANSPARÊNCIA - DADOS ATRICON")
print("=" * 100)

print(f"\nTotal de municípios analisados: {len(df_analise)}")
print(f"Municípios com dados ATRICON: {df_analise['indice_final'].notna().sum()}")

if df_analise['indice_final'].notna().sum() > 0:
    print(f"\nÍndice médio de transparência: {df_analise['indice_final'].mean():.2f}%")
    print(f"Índice mediano de transparência: {df_analise['indice_final'].median():.2f}%")
    print(f"Desvio padrão: {df_analise['indice_final'].std():.2f}%")
    
    print(f"\nMaior índice: {df_analise['indice_final'].max():.2f}% - {df_analise.loc[df_analise['indice_final'].idxmax(), 'Municipio']}")
    print(f"Menor índice: {df_analise['indice_final'].min():.2f}% - {df_analise.loc[df_analise['indice_final'].idxmin(), 'Municipio']}")

print("\n" + "=" * 100)
print("OBSERVAÇÕES IMPORTANTES")
print("=" * 100)
print("""
1. Os dados ATRICON fornecem uma avaliação objetiva da transparência pública municipal
2. O índice final considera tanto a avaliação inicial quanto a validação dos dados
3. A classificação em níveis (Diamante, Ouro, Prata, etc.) facilita a interpretação
4. A análise de correlação permite identificar relações entre transparência e indicadores socioeconômicos

Próximos passos:
- Notebook 3: Análise detalhada de transparência e conformidade com normas
- Notebook 4: Análise comparativa e identificação de padrões
""")

In [ ]:
# Salvar dataset final com dados de transparência
df_analise.to_csv('municipios_com_transparencia.csv', index=False, encoding='utf-8-sig')
print("\nDataset final salvo: municipios_com_transparencia.csv")
print("\nPróximo passo: Notebook 3 - Análise de Transparência")